In [1]:
import pandas as pd
import os
import random
from tqdm import tqdm
import pickle
from sklearn.preprocessing import MinMaxScaler

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
#TODO when i am filling values from current index back to a length of window size, i didn't consider the previous interaction. this has to be updated.

In [3]:
regenerate_context_data = True
sequence_augmentation = True
whole_session_context = False
model_test_run = False
data_autoencoder = False
pad_to_window_size = True

PATH_TO_LOAD = '../data/04_Merged'
combined_context_path = '../data/05_Interaction_Sequences/context.csv'

window = 100 #seconds

base_path = '../datasets/sequential/'
augmentation_folder = 'aug/' if sequence_augmentation else 'non_aug/'
if model_test_run:
    augmentation_folder = 'test/aug/' if sequence_augmentation else 'test/non_aug/'

sequence_context_path = f'{base_path}{augmentation_folder}parameters/sequence_context.csv'
parameter_path = f'{base_path}{augmentation_folder}parameters'
train_session_path = f'{base_path}{augmentation_folder}parameters/train_sessions.pkl'
test_session_path = f'{base_path}{augmentation_folder}parameters/test_sessions.pkl'
train_dynamic_context_path = f'{base_path}{augmentation_folder}dynamic_context/train.csv'
test_dynamic_context_path = f'{base_path}{augmentation_folder}dynamic_context/test.csv'
train_static_context_path = f'{base_path}{augmentation_folder}static_context/train.csv'
test_static_context_path = f'{base_path}{augmentation_folder}static_context/test.csv'

In [4]:
all_columns = ['index', 'avg_irradiation', 'steering_speed', 'temperature_out', 'hour',
       'month', 'odometer', 'light_sensor_rear', 'light_sensor_front',
       'temperature_in', 'KBI_speed', 'soc', 'ESP_speed', 'latitude',
       'longitude', 'seatbelt_codriver', 'seatbelt_rear_l', 'seatbelt_rear_m',
       'seatbelt_rear_r', 'CHA_ESP_drive_mode', 'CHA_MO_drive_mode',
       'rain_sensor', 'street_category', 'kickdown', 'altitude',
       'driving_program', 'datetime', 'session', 'Label', 'ID',
       'FunctionValue', 'domain', 'BeginTime', 'time_second',
       'distance_driven', 'ts_normalized', 'weekday']

selected = [ 'avg_irradiation', 'steering_speed', 'temperature_out', 'hour',
       'month', 'light_sensor_rear', 'light_sensor_front',
       'temperature_in', 'KBI_speed', 'soc', 'latitude',
       'longitude', 'seatbelt_codriver', 'seatbelt_rear_l',
       'seatbelt_rear_r', 'street_category', 'altitude',
       'datetime', 'session', 'time_second',
       'distance_driven', 'weekday'
]

bad_quality = ['CHA_ESP_drive_mode', 
             'CHA_MO_drive_mode',
             'rain_sensor',
             'kickdown',
             'ESP_speed',
             'seatbelt_rear_m',
            'driving_program',
            'ts_normalized'
             ]

dynamic_context_var = ['avg_irradiation', 'steering_speed', 'temperature_out', 
                       'light_sensor_rear', 'light_sensor_front', 
                       'temperature_in', 'KBI_speed', 'soc', 'latitude',
                       'longitude',  'street_category', 'altitude','time_second',
                       'distance_driven']
static_context_var = ['car_id', 'month', 'weekday', 'hour', 'seatbelt_codriver', 'seatbelt_rear_l',
                       'seatbelt_rear_r',]
#Todo take average of these value over a window
constant_context_var = ['avg_irradiation','temperature_out'] #to be filled

In [5]:
def load_context(vehicle):
    context_lists = dynamic_context_var + static_context_var + ['session', 'datetime', 'Label']
    context_lists.remove('car_id')
    df = pd.read_csv(os.path.join(PATH_TO_LOAD, vehicle + "_merged.csv"), parse_dates=['datetime'], usecols=context_lists, low_memory=False)
    df = df.sort_values(by=['session','datetime'])
    return df

vehicles = ['SEB880','SEB882','SEB883','SEB885','SEB888','SEB889']
context_data_list = []
for vehicle in tqdm(vehicles):
    context_curr = load_context(vehicle)
    context_curr['car_id'] = vehicle
    context_data_list.append(context_curr)
context_data = pd.concat(context_data_list, axis=0).reset_index()
context_data.to_csv(combined_context_path, index=False)

100%|██████████| 6/6 [00:53<00:00,  8.85s/it]


In [6]:
context_data = pd.read_csv(combined_context_path, parse_dates=['datetime'], low_memory=False)
vehicle_list = context_data.car_id.unique().tolist()
random.shuffle(vehicle_list)
vehicle_dict = {}
used_values = set()
for vehicle in vehicle_list:
    value = random.randint(1, len(vehicle_list))
    while value in used_values:
        value = random.randint(1, len(vehicle_list))
    vehicle_dict[vehicle] = value
    used_values.add(value)

context_data['car_id'] = context_data['car_id'].map(vehicle_dict)
context_data = context_data.sort_values(by=['session','datetime'])

In [7]:
grouped_columns = ['session', 'datetime']
columns_to_average = dynamic_context_var + static_context_var
grouped_data = context_data.groupby(grouped_columns)[columns_to_average].mean().reset_index()
merged_data = pd.merge(context_data[['session', 'datetime', 'Label']], grouped_data, on=grouped_columns, suffixes=('', '_mean'))

# context_data['session'] = context_data['session'].astype(int)
# context_data = context_data[dynamic_context_var + static_context_var + ['session', 'datetime', 'Label']]
# context_data = context_data.groupby(['session', 'datetime'])[dynamic_context_var + static_context_var].mean().reset_index()

In [8]:
context_data['Label'] = context_data['Label'].replace('car/driveMode/0', 'car/driveMode/0.0')
context_data['Label'] = context_data['Label'].replace('car/driveMode/2', 'car/driveMode/2.0')
context_data['Label'] = context_data['Label'].replace('car/driveMode/3', 'car/driveMode/3.0')

# context_data['Label'] = context_data['Label'].replace('car/charismaLevel/Abgesenkt', 'car/charismaLevel/change')
# context_data['Label'] = context_data['Label'].replace('car/charismaLevel/Lift', 'car/charismaLevel/change')
# context_data['Label'] = context_data['Label'].replace('car/charismaLevel/Mittel', 'car/charismaLevel/change')
# context_data['Label'] = context_data['Label'].replace('car/charismaLevel/Tief', 'car/charismaLevel/change')

In [9]:
context_data.columns

Index(['index', 'avg_irradiation', 'steering_speed', 'temperature_out', 'hour',
       'month', 'light_sensor_rear', 'light_sensor_front', 'temperature_in',
       'KBI_speed', 'soc', 'latitude', 'longitude', 'seatbelt_codriver',
       'seatbelt_rear_l', 'seatbelt_rear_r', 'street_category', 'altitude',
       'datetime', 'session', 'Label', 'time_second', 'distance_driven',
       'weekday', 'car_id'],
      dtype='object')

In [10]:
cleaned_list = [x for x in context_data.Label.unique().tolist() if not pd.isna(x)]
sorted(cleaned_list)

['car/ESS/on',
 'car/Start/ParkAssistant',
 'car/charismaLevel/Abgesenkt',
 'car/charismaLevel/Lift',
 'car/charismaLevel/Mittel',
 'car/charismaLevel/Tief',
 'car/driveMode/0.0',
 'car/driveMode/1.0',
 'car/driveMode/2.0',
 'car/driveMode/3.0',
 'clima/AC/ECO',
 'clima/AC/off',
 'clima/AC/on',
 'media/selectedSource/Bluetooth',
 'media/selectedSource/CarPlay',
 'media/selectedSource/Favorite',
 'media/selectedSource/Radio',
 'navi/Start/Address',
 'navi/Start/Favorite',
 'phone/Call/Favorite',
 'phone/Call/PersonX',
 'phone/Connect/NewDevice',
 'phone/Start/AndroidAuto',
 'phone/Start/CarPlay',
 'phone/goTo/Favorite']

In [11]:
# TODO ess status - Untill the point ess is swtiched on, add a status that ess is off now - 1 on/ 0 off
# current drive mode - 0, 1, 2, 3
# current AC mode - Eco, on, off
# current selected source - bluetooth, carplay, favourite, radio, no source
# navigation guidance activation status - 0 - inactive/ 1 - active
# phone call status - 
# os of the phone connected
# new bluetooth device around status - 1 - yes and 0- no
# phone connected or not -  (needed to make a phone call)
# time since last interaction as a static variable
# convert street category into text highway, countryside etc
# season
# ventilation running
# racetrack proximity
# desired temperature - this can go in user embedding may ?
# user embedding is introduced as kind of memory
# current car temperature

In [12]:
context_data[context_data['Label']=='car/ESS/on']

,index,avg_irradiation,steering_speed,temperature_out,hour,month,light_sensor_rear,light_sensor_front,temperature_in,KBI_speed,...,seatbelt_rear_r,street_category,altitude,datetime,session,Label,time_second,distance_driven,weekday,car_id
146831,146814,0.000000,0.000000,12.0,20,12,0.000000,0.000000,21.500000,22.037324,...,1.0,0.0,194.000000,2022-12-29 20:23:44,205.0,car/ESS/on,870.0,14728.0,3,1
147046,147029,0.000000,0.000000,12.0,20,12,0.000000,0.000000,22.000000,0.000000,...,1.0,7.0,198.000000,2022-12-29 20:27:19,205.0,car/ESS/on,1085.0,14729.0,3,1
3744925,1690173,6.419186,0.000000,9.5,10,12,4800.000000,1350.000000,26.000000,0.000000,...,0.0,7.0,474.000000,2022-12-22 10:31:00,1659.0,car/ESS/on,9614.0,28353.0,3,6
4334549,2279768,0.000000,0.000000,-3.0,9,2,0.000000,72.000000,22.500000,0.000000,...,0.0,7.0,338.000000,2023-02-07 09:05:58,1908.0,car/ESS/on,7219.0,34029.0,1,6
4574089,2519286,0.266636,0.000000,0.0,13,2,14398.797565,6041.855708,23.181415,0.000000,...,0.0,0.0,468.000000,2023-02-28 13:49:30,1997.0,car/ESS/on,22299.0,36521.0,1,6
5540311,348153,60.000000,5.029246,14.5,10,10,17735.744925,5208.488347,28.000000,51.200000,...,0.0,7.0,694.000000,2022-10-21 10:56:37,2587.0,car/ESS/on,18495.0,10472.0,4,4
5540322,348164,50.000000,0.000000,14.5,10,10,18000.000000,5890.053078,28.000000,58.240000,...,0.0,4.0,688.000000,2022-10-21 10:56:48,2587.0,car/ESS/on,18506.0,10472.0,4,4
5543739,351581,70.000000,0.000000,13.0,11,10,22000.000000,4650.000000,28.000000,83.840000,...,0.0,5.0,931.706949,2022-10-21 11:53:44,2587.0,car/ESS/on,21922.0,10565.0,4,4
9930082,1165732,10.000000,0.000000,2.0,17,2,5200.000000,2153.365494,6.500000,67.203988,...,0.0,2.0,263.814125,2023-02-26 17:30:43,5256.0,car/ESS/on,270.0,4.0,6,2


In [13]:
## adding the current status of ess as a context variable.
context_data = context_data.sort_index()
ess_info = []
for session in tqdm(context_data.session.unique().tolist()):
    df = context_data[context_data['session']== session].copy()
    df['ess_status'] = 0
    ess_index = df.index[df['Label'] == 'car/ESS/on']
    for idx in ess_index:
        df.loc[:idx, 'ess_status'] = 0  # Set 'ess_status' to 0 for rows before the 'car/ESS/on' row
        df.loc[idx:, 'ess_status'] = 1
    ess_info.append(df)
data_ess = pd.concat(ess_info, axis=0)

100%|██████████| 1704/1704 [00:15<00:00, 109.75it/s]


In [14]:
data_ess[data_ess['session']==205][['Label', 'session', 'ess_status']].dropna()

,Label,session,ess_status
146086,media/selectedSource/Radio,205.0,0
146205,car/driveMode/1.0,205.0,0
146621,car/charismaLevel/Lift,205.0,0
146634,car/charismaLevel/Abgesenkt,205.0,0
146831,car/ESS/on,205.0,0
147046,car/ESS/on,205.0,1
147122,navi/Start/Address,205.0,1
147354,car/driveMode/0.0,205.0,1
147392,media/selectedSource/Radio,205.0,1
147566,media/selectedSource/Bluetooth,205.0,1


In [15]:
data_ess[data_ess['session']==1645][['Label', 'session', 'ess_status']].dropna()

,Label,session,ess_status
3697821,car/driveMode/3.0,1645.0,0
3698358,car/driveMode/0.0,1645.0,0
3698581,car/driveMode/3.0,1645.0,0
3699191,car/driveMode/0.0,1645.0,0
3699399,car/driveMode/3.0,1645.0,0
3699483,media/selectedSource/Bluetooth,1645.0,0
3699484,media/selectedSource/Radio,1645.0,0
3699840,car/driveMode/0.0,1645.0,0


In [16]:
test_session = [1645]

drive_mode_info = []
for session in tqdm(data_ess.session.unique().tolist()):
# for session in tqdm(test_session):
    df = data_ess[data_ess['session']== session].copy()
    df = df.sort_values(by=['datetime'])
    df['current_drive_mode'] = 'car/driveMode/0.0'
    drive_modes = {'car/driveMode/0.0', 'car/driveMode/1.0', 'car/driveMode/2.0', 'car/driveMode/3.0'}

    # Iterate over DataFrame
    for i, row in df.iterrows():
        label = row['Label']
        if label in drive_modes:
            df.at[i, 'current_drive_mode'] = label
            df.loc[i:, 'current_drive_mode'] = label
        else:
            continue

    drive_mode_info.append(df)
data_drivemode = pd.concat(drive_mode_info, axis=0)

100%|██████████| 1704/1704 [08:04<00:00,  3.52it/s]


In [17]:
data_drivemode[data_drivemode['session']==16][['Label', 'session', 'current_drive_mode']].dropna()

,Label,session,current_drive_mode
4420,media/selectedSource/Radio,16.0,car/driveMode/0.0
4421,media/selectedSource/Radio,16.0,car/driveMode/0.0
5035,media/selectedSource/Bluetooth,16.0,car/driveMode/0.0
6338,media/selectedSource/Radio,16.0,car/driveMode/0.0
7484,media/selectedSource/Radio,16.0,car/driveMode/0.0


In [18]:
data_drivemode.Label.unique().tolist()

[nan,
 'media/selectedSource/Bluetooth',
 'media/selectedSource/Radio',
 'media/selectedSource/Favorite',
 'navi/Start/Address',
 'navi/Start/Favorite',
 'phone/goTo/Favorite',
 'car/driveMode/0.0',
 'car/driveMode/2.0',
 'phone/Start/CarPlay',
 'clima/AC/on',
 'car/driveMode/1.0',
 'car/charismaLevel/Lift',
 'car/charismaLevel/Abgesenkt',
 'car/ESS/on',
 'car/driveMode/3.0',
 'phone/Start/AndroidAuto',
 'clima/AC/off',
 'car/charismaLevel/Tief',
 'phone/Connect/NewDevice',
 'phone/Call/Favorite',
 'phone/Call/PersonX',
 'clima/AC/ECO',
 'car/charismaLevel/Mittel',
 'car/Start/ParkAssistant',
 'media/selectedSource/CarPlay']

In [19]:
data_drivemode.dropna()[data_drivemode.dropna()['Label'].str.contains('clima')]['Label'].unique().tolist()

['clima/AC/on', 'clima/AC/off', 'clima/AC/ECO']

In [20]:
# import matplotlib.pyplot as plt

# df = data_drivemode[['temperature_out', 'temperature_in', 'month', 'car_id']]
# df['month'] = pd.Categorical(df['month'], categories=df['month'].unique().tolist(), ordered=True)
# averages = df.groupby(['car_id', 'month']).mean().reset_index()

# # Plotting
# fig, axes = plt.subplots(nrows=len(df['car_id'].unique()), ncols=1, figsize=(10, 6 * len(df['car_id'].unique())))

# for i, (car_id, group) in enumerate(averages.groupby('car_id')):
#     ax = axes[i]
#     group.plot(x='month', y=['temperature_out', 'temperature_in'], kind='bar', ax=ax, title=f'Car ID: {car_id}', legend=True)
#     ax.set_ylabel('Temperature')
#     ax.set_xlabel('Month')
#     ax.legend(['Temperature Out', 'Temperature In'])

# plt.tight_layout()
# plt.show()

In [21]:
data_drivemode['session'].unique().tolist()

[7.0,
 16.0,
 20.0,
 25.0,
 27.0,
 33.0,
 35.0,
 40.0,
 42.0,
 45.0,
 46.0,
 50.0,
 52.0,
 76.0,
 94.0,
 150.0,
 151.0,
 163.0,
 164.0,
 172.0,
 173.0,
 177.0,
 178.0,
 183.0,
 186.0,
 188.0,
 189.0,
 191.0,
 192.0,
 195.0,
 196.0,
 197.0,
 199.0,
 202.0,
 203.0,
 205.0,
 206.0,
 209.0,
 210.0,
 211.0,
 213.0,
 252.0,
 253.0,
 255.0,
 256.0,
 258.0,
 270.0,
 274.0,
 276.0,
 277.0,
 357.0,
 385.0,
 387.0,
 389.0,
 397.0,
 401.0,
 404.0,
 405.0,
 406.0,
 408.0,
 411.0,
 413.0,
 416.0,
 418.0,
 420.0,
 422.0,
 425.0,
 426.0,
 432.0,
 433.0,
 434.0,
 435.0,
 439.0,
 440.0,
 441.0,
 444.0,
 447.0,
 451.0,
 453.0,
 456.0,
 457.0,
 459.0,
 460.0,
 461.0,
 465.0,
 466.0,
 470.0,
 474.0,
 476.0,
 479.0,
 481.0,
 483.0,
 484.0,
 488.0,
 489.0,
 490.0,
 491.0,
 492.0,
 493.0,
 496.0,
 500.0,
 502.0,
 503.0,
 504.0,
 509.0,
 510.0,
 513.0,
 514.0,
 515.0,
 516.0,
 519.0,
 520.0,
 522.0,
 523.0,
 524.0,
 526.0,
 527.0,
 532.0,
 535.0,
 538.0,
 541.0,
 542.0,
 543.0,
 544.0,
 548.0,
 550.0,
 553.0,


In [22]:
data_drivemode.dropna()[data_drivemode.dropna()['Label'].str.contains('clima')].dropna()

,index,avg_irradiation,steering_speed,temperature_out,hour,month,light_sensor_rear,light_sensor_front,temperature_in,KBI_speed,...,altitude,datetime,session,Label,time_second,distance_driven,weekday,car_id,ess_status,current_drive_mode
123718,123701,0.000000,0.000000,5.0,21,12,0.000000,0.0,19.000000,0.000000,...,195.385842,2022-12-27 21:37:25,192.0,clima/AC/on,58.0,14604.0,1,1,0,car/driveMode/0.0
229147,229117,186.359989,0.000000,3.0,10,4,15200.000000,6126.0,26.500000,0.000000,...,410.000000,2023-04-05 10:18:00,387.0,clima/AC/off,3028.0,22267.0,2,1,0,car/driveMode/3.0
511269,511182,310.000000,0.000000,16.0,13,4,101200.000000,6126.0,30.000000,68.480000,...,430.612621,2023-04-27 13:33:32,509.0,clima/AC/off,337.0,24107.0,3,1,0,car/driveMode/0.0
924794,924646,130.000000,10.908400,17.0,8,5,9200.000000,4752.0,23.701887,30.663599,...,306.000000,2023-05-31 08:03:30,735.0,clima/AC/off,1157.0,26980.0,2,1,0,car/driveMode/0.0
1086117,1085927,30.000000,5.000000,20.0,19,6,6400.000000,2832.0,28.000000,12.959916,...,716.000000,2023-06-04 19:43:36,766.0,clima/AC/off,3007.0,27591.0,6,1,0,car/driveMode/0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9637092,872801,31.480222,0.000000,1.0,11,1,21194.333573,6126.0,23.000000,142.080000,...,102.000000,2023-01-20 11:32:39,5086.0,clima/AC/ECO,1075.0,16400.0,4,2,0,car/driveMode/0.0
9859588,1095258,0.000000,36.552309,0.5,1,2,0.000000,0.0,9.000000,15.360000,...,346.601038,2023-02-05 01:13:43,5169.0,clima/AC/on,178.0,18822.0,6,2,0,car/driveMode/0.0
9916990,1152649,0.000000,0.000000,12.0,18,2,0.000000,102.0,25.000000,0.000000,...,280.000000,2023-02-23 18:55:01,5246.0,clima/AC/ECO,1497.0,19744.0,3,2,0,car/driveMode/0.0
9916991,1360815,0.000000,0.000000,12.0,18,2,0.000000,102.0,25.000000,0.000000,...,280.000000,2023-02-23 18:55:01,5246.0,clima/AC/ECO,1497.0,19744.0,3,2,0,car/driveMode/0.0


In [23]:
data_drivemode[data_drivemode['session']==387].sort_index()

,index,avg_irradiation,steering_speed,temperature_out,hour,month,light_sensor_rear,light_sensor_front,temperature_in,KBI_speed,...,altitude,datetime,session,Label,time_second,distance_driven,weekday,car_id,ess_status,current_drive_mode
226118,226092,0.0,0.0,0.0,9,4,0.0,0.000000,0.0,0.0,...,0.0,2023-04-05 09:27:32,387.0,NaN,0.0,0.0,2,1,0,car/driveMode/0.0
226119,226093,0.0,0.0,0.0,9,4,3200.0,4597.346973,0.0,0.0,...,0.0,2023-04-05 09:27:33,387.0,NaN,1.0,22245.0,2,1,0,car/driveMode/0.0
226120,226094,0.0,0.0,2.0,9,4,3200.0,5858.598660,0.0,0.0,...,0.0,2023-04-05 09:27:34,387.0,NaN,2.0,22245.0,2,1,0,car/driveMode/0.0
226121,226095,0.0,0.0,2.0,9,4,3200.0,5910.000000,1.5,0.0,...,0.0,2023-04-05 09:27:35,387.0,NaN,3.0,22245.0,2,1,0,car/driveMode/0.0
226122,226096,0.0,0.0,2.0,9,4,3200.0,5910.000000,1.5,0.0,...,0.0,2023-04-05 09:27:35,387.0,NaN,3.0,22245.0,2,1,0,car/driveMode/0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230913,230883,290.0,0.0,4.0,10,4,95200.0,6126.000000,21.0,0.0,...,474.0,2023-04-05 10:47:27,387.0,NaN,4795.0,22282.0,2,1,0,car/driveMode/0.0
230914,230884,290.0,0.0,4.0,10,4,95200.0,6126.000000,21.0,0.0,...,474.0,2023-04-05 10:47:28,387.0,NaN,4796.0,22282.0,2,1,0,car/driveMode/0.0
230915,230885,290.0,0.0,4.0,10,4,95200.0,6126.000000,21.0,0.0,...,474.0,2023-04-05 10:47:29,387.0,NaN,4797.0,22282.0,2,1,0,car/driveMode/0.0
230916,230886,290.0,0.0,4.0,10,4,95200.0,6126.000000,21.0,0.0,...,474.0,2023-04-05 10:47:30,387.0,NaN,4798.0,22282.0,2,1,0,car/driveMode/0.0


In [24]:
test_session = [387]

clima_mode_info = []
temp_delta = []
for session in tqdm(data_drivemode.session.unique().tolist()):
# for session in tqdm(test_session):
    df = data_drivemode[data_drivemode['session']== session].copy().sort_index().reset_index()
    filtered_df = df.dropna()[df.dropna()['Label'].str.contains('clima')].dropna()
    if filtered_df.empty:
        df['current_clima_mode']  = 'clima/AC/Auto'
    else:
        df = df.sort_values(by=['datetime'])
        df['current_clima_mode'] = 'clima/AC/off'
        clima_modes = {'clima/AC/on', 'clima/AC/off', 'clima/AC/ECO'}

        # Iterate over DataFrame
        first_clima_interaction_index = 0
        for i, row in df.iterrows():
            label = row['Label']
            if label in clima_modes:
                if first_clima_interaction_index == 0:
                    first_clima_interaction_index = i
                df.at[i, 'current_clima_mode'] = label
                df.loc[i:, 'current_clima_mode'] = label
            else:
                continue

        if first_clima_interaction_index is not None:
            temp1_index = df[df['temperature_in'] != 0].index.min()
            temp1 = df.at[temp1_index, 'temperature_in']
            temp2 = df['temperature_in'].iloc[first_clima_interaction_index - 1]
            # print(temp1-temp2)
            if abs(float(temp1) - float(temp2)) > 3:
                # print('in1')
                if df.at[first_clima_interaction_index, 'Label'] == 'clima/AC/ECO':
                    df.loc[:first_clima_interaction_index - 1, 'current_clima_mode']  = 'clima/AC/on'
                elif df.at[first_clima_interaction_index, 'Label'] == 'clima/AC/on':
                    df.loc[:first_clima_interaction_index - 1, 'current_clima_mode']  = 'clima/AC/ECO'
                elif df.at[first_clima_interaction_index, 'Label'] == 'clima/AC/off':
                    df.loc[:first_clima_interaction_index - 1, 'current_clima_mode']  = 'clima/AC/ECO'
            else:
                df.loc[:first_clima_interaction_index - 1, 'current_clima_mode'] = 'clima/AC/off'
    clima_mode_info.append(df)
data_clima = pd.concat(clima_mode_info, axis=0)

100%|██████████| 1704/1704 [00:43<00:00, 39.49it/s]


In [25]:
data_clima

,level_0,index,avg_irradiation,steering_speed,temperature_out,hour,month,light_sensor_rear,light_sensor_front,temperature_in,...,datetime,session,Label,time_second,distance_driven,weekday,car_id,ess_status,current_drive_mode,current_clima_mode
0,0,0,0.000000,0.000000,0.0,20,9,0.0,0.000000,0.0,...,2022-09-07 20:08:33,7.0,NaN,0.0,0.0,2,1,0,car/driveMode/0.0,clima/AC/Auto
1,1,1,0.000000,0.000000,0.0,20,9,0.0,0.000000,0.0,...,2022-09-07 20:08:34,7.0,NaN,1.0,10845.0,2,1,0,car/driveMode/0.0,clima/AC/Auto
2,2,2,0.000000,0.000000,25.0,20,9,0.0,0.000000,0.0,...,2022-09-07 20:08:35,7.0,NaN,2.0,10845.0,2,1,0,car/driveMode/0.0,clima/AC/Auto
3,3,3,0.000000,0.000000,25.0,20,9,0.0,0.000000,27.0,...,2022-09-07 20:08:36,7.0,NaN,3.0,10845.0,2,1,0,car/driveMode/0.0,clima/AC/Auto
4,4,4,0.000000,0.000000,25.0,20,9,0.0,0.000000,27.0,...,2022-09-07 20:08:36,7.0,NaN,3.0,10845.0,2,1,0,car/driveMode/0.0,clima/AC/Auto
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2066,10124879,1360504,53.755324,0.000000,13.5,17,3,17600.0,6126.000000,28.5,...,2023-03-31 17:09:13,5399.0,NaN,5360.0,146.0,4,2,0,car/driveMode/0.0,clima/AC/Auto
2067,10124880,1360505,50.000000,0.000000,13.5,17,3,17200.0,5920.781157,28.5,...,2023-03-31 17:09:14,5399.0,NaN,5361.0,146.0,4,2,0,car/driveMode/0.0,clima/AC/Auto
2068,10124881,1360506,56.226313,15.000000,13.5,17,3,18000.0,5699.908827,28.5,...,2023-03-31 17:09:15,5399.0,NaN,5362.0,146.0,4,2,0,car/driveMode/0.0,clima/AC/Auto
2069,10124882,1360507,60.000000,0.000000,13.5,17,3,17600.0,5437.218229,28.5,...,2023-03-31 17:09:16,5399.0,NaN,5363.0,146.0,4,2,0,car/driveMode/0.0,clima/AC/Auto


In [26]:
data_clima.dropna()[data_clima.dropna()['Label'].str.contains('selectedSource')].dropna()['Label'].unique()

array(['media/selectedSource/Bluetooth', 'media/selectedSource/Radio',
       'media/selectedSource/Favorite', 'media/selectedSource/CarPlay'],
      dtype=object)

In [27]:
data_clima[data_clima['session']==16].Label.dropna()

233         media/selectedSource/Radio
234         media/selectedSource/Radio
848     media/selectedSource/Bluetooth
2151        media/selectedSource/Radio
3297        media/selectedSource/Radio
Name: Label, dtype: object

In [28]:
test_session = [387]

data_clima = data_clima.sort_values(by=['session', 'datetime'])
data_clima = data_clima.reset_index(drop=True)
data_clima = data_clima.drop(columns=['level_0', 'index'])

media_source_info = []
for session in tqdm(data_clima.session.unique().tolist()):
    df = data_clima[data_clima['session']== session].copy().sort_index().reset_index()
    df = df.sort_values(by=['datetime'])
    df['current_media_source'] = 'media/selectedSource/unavailable'
    media_source = {'media/selectedSource/Bluetooth', 'media/selectedSource/Radio',
    'media/selectedSource/Favorite', 'media/selectedSource/CarPlay'}

    for i, row in df.iterrows():
        label = row['Label']
        if label in clima_modes:
            df.at[i, 'current_media_source'] = label
            df.loc[i:, 'current_media_source'] = label
        else:
            continue
    media_source_info.append(df)
data_media = pd.concat(media_source_info, axis=0)

100%|██████████| 1704/1704 [07:42<00:00,  3.68it/s]


In [29]:
data_media

,index,avg_irradiation,steering_speed,temperature_out,hour,month,light_sensor_rear,light_sensor_front,temperature_in,KBI_speed,...,session,Label,time_second,distance_driven,weekday,car_id,ess_status,current_drive_mode,current_clima_mode,current_media_source
0,0,0.000000,0.000000,0.0,20,9,0.0,0.000000,0.0,0.00,...,7.0,NaN,0.0,0.0,2,1,0,car/driveMode/0.0,clima/AC/Auto,media/selectedSource/unavailable
1,1,0.000000,0.000000,0.0,20,9,0.0,0.000000,0.0,0.00,...,7.0,NaN,1.0,10845.0,2,1,0,car/driveMode/0.0,clima/AC/Auto,media/selectedSource/unavailable
2,2,0.000000,0.000000,25.0,20,9,0.0,0.000000,0.0,0.00,...,7.0,NaN,2.0,10845.0,2,1,0,car/driveMode/0.0,clima/AC/Auto,media/selectedSource/unavailable
3,3,0.000000,0.000000,25.0,20,9,0.0,0.000000,27.0,0.00,...,7.0,NaN,3.0,10845.0,2,1,0,car/driveMode/0.0,clima/AC/Auto,media/selectedSource/unavailable
4,4,0.000000,0.000000,25.0,20,9,0.0,0.000000,27.0,0.00,...,7.0,NaN,3.0,10845.0,2,1,0,car/driveMode/0.0,clima/AC/Auto,media/selectedSource/unavailable
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2066,10124879,53.755324,0.000000,13.5,17,3,17600.0,6126.000000,28.5,92.48,...,5399.0,NaN,5360.0,146.0,4,2,0,car/driveMode/0.0,clima/AC/Auto,media/selectedSource/unavailable
2067,10124880,50.000000,0.000000,13.5,17,3,17200.0,5920.781157,28.5,93.12,...,5399.0,NaN,5361.0,146.0,4,2,0,car/driveMode/0.0,clima/AC/Auto,media/selectedSource/unavailable
2068,10124881,56.226313,15.000000,13.5,17,3,18000.0,5699.908827,28.5,93.44,...,5399.0,NaN,5362.0,146.0,4,2,0,car/driveMode/0.0,clima/AC/Auto,media/selectedSource/unavailable
2069,10124882,60.000000,0.000000,13.5,17,3,17600.0,5437.218229,28.5,93.76,...,5399.0,NaN,5363.0,146.0,4,2,0,car/driveMode/0.0,clima/AC/Auto,media/selectedSource/unavailable


In [30]:
data_media.dropna()[data_media.dropna()['Label'].str.contains('navi')].dropna()

,index,avg_irradiation,steering_speed,temperature_out,hour,month,light_sensor_rear,light_sensor_front,temperature_in,KBI_speed,...,session,Label,time_second,distance_driven,weekday,car_id,ess_status,current_drive_mode,current_clima_mode,current_media_source
183,13611,0.0,0.000000,23.5,17,9,1600.000000,3221.903447,29.115677,27.84,...,33.0,navi/Start/Address,181.0,11198.0,0,1,0,car/driveMode/0.0,clima/AC/Auto,media/selectedSource/unavailable
182,13610,0.0,0.000000,23.5,17,9,1600.000000,3221.903447,29.115677,27.84,...,33.0,navi/Start/Favorite,181.0,11198.0,0,1,0,car/driveMode/0.0,clima/AC/Auto,media/selectedSource/unavailable
1776,21483,150.0,0.000000,16.0,7,9,1600.000000,1416.000000,24.606187,112.96,...,35.0,navi/Start/Address,1775.0,11257.0,1,1,0,car/driveMode/0.0,clima/AC/Auto,media/selectedSource/unavailable
1775,21482,150.0,0.000000,16.0,7,9,1600.000000,1416.000000,24.606187,112.96,...,35.0,navi/Start/Favorite,1775.0,11257.0,1,1,0,car/driveMode/0.0,clima/AC/Auto,media/selectedSource/unavailable
784,23812,0.0,0.000000,20.0,18,9,1600.000000,1195.217862,25.000000,0.00,...,40.0,navi/Start/Favorite,783.0,11351.0,2,1,0,car/driveMode/0.0,clima/AC/Auto,media/selectedSource/unavailable
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
726,10111789,0.0,3.955310,13.5,8,3,1600.000000,512.863260,21.000000,80.32,...,5364.0,navi/Start/Favorite,1505.0,22519.0,3,2,0,car/driveMode/0.0,clima/AC/Auto,media/selectedSource/unavailable
727,10111790,0.0,3.955310,13.5,8,3,1600.000000,512.863260,21.000000,80.32,...,5364.0,navi/Start/Favorite,1505.0,22519.0,3,2,0,car/driveMode/0.0,clima/AC/Auto,media/selectedSource/unavailable
728,10111791,0.0,3.955310,13.5,8,3,1600.000000,512.863260,21.000000,80.32,...,5364.0,navi/Start/Address,1505.0,22519.0,3,2,0,car/driveMode/0.0,clima/AC/Auto,media/selectedSource/unavailable
729,10111792,0.0,3.955310,13.5,8,3,1600.000000,512.863260,21.000000,80.32,...,5364.0,navi/Start/Address,1505.0,22519.0,3,2,0,car/driveMode/0.0,clima/AC/Auto,media/selectedSource/unavailable


In [31]:
data_media.dropna()[data_media.dropna()['Label'].str.contains('navi')].dropna()['Label'].unique().tolist()

['navi/Start/Address', 'navi/Start/Favorite']

In [32]:
data_media

,index,avg_irradiation,steering_speed,temperature_out,hour,month,light_sensor_rear,light_sensor_front,temperature_in,KBI_speed,...,session,Label,time_second,distance_driven,weekday,car_id,ess_status,current_drive_mode,current_clima_mode,current_media_source
0,0,0.000000,0.000000,0.0,20,9,0.0,0.000000,0.0,0.00,...,7.0,NaN,0.0,0.0,2,1,0,car/driveMode/0.0,clima/AC/Auto,media/selectedSource/unavailable
1,1,0.000000,0.000000,0.0,20,9,0.0,0.000000,0.0,0.00,...,7.0,NaN,1.0,10845.0,2,1,0,car/driveMode/0.0,clima/AC/Auto,media/selectedSource/unavailable
2,2,0.000000,0.000000,25.0,20,9,0.0,0.000000,0.0,0.00,...,7.0,NaN,2.0,10845.0,2,1,0,car/driveMode/0.0,clima/AC/Auto,media/selectedSource/unavailable
3,3,0.000000,0.000000,25.0,20,9,0.0,0.000000,27.0,0.00,...,7.0,NaN,3.0,10845.0,2,1,0,car/driveMode/0.0,clima/AC/Auto,media/selectedSource/unavailable
4,4,0.000000,0.000000,25.0,20,9,0.0,0.000000,27.0,0.00,...,7.0,NaN,3.0,10845.0,2,1,0,car/driveMode/0.0,clima/AC/Auto,media/selectedSource/unavailable
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2066,10124879,53.755324,0.000000,13.5,17,3,17600.0,6126.000000,28.5,92.48,...,5399.0,NaN,5360.0,146.0,4,2,0,car/driveMode/0.0,clima/AC/Auto,media/selectedSource/unavailable
2067,10124880,50.000000,0.000000,13.5,17,3,17200.0,5920.781157,28.5,93.12,...,5399.0,NaN,5361.0,146.0,4,2,0,car/driveMode/0.0,clima/AC/Auto,media/selectedSource/unavailable
2068,10124881,56.226313,15.000000,13.5,17,3,18000.0,5699.908827,28.5,93.44,...,5399.0,NaN,5362.0,146.0,4,2,0,car/driveMode/0.0,clima/AC/Auto,media/selectedSource/unavailable
2069,10124882,60.000000,0.000000,13.5,17,3,17600.0,5437.218229,28.5,93.76,...,5399.0,NaN,5363.0,146.0,4,2,0,car/driveMode/0.0,clima/AC/Auto,media/selectedSource/unavailable


In [33]:
data_media = data_media.sort_values(by=['session', 'datetime'])
data_media = data_media.reset_index(drop=True)
data_media = data_media.drop(columns=['index'])
data_media

,avg_irradiation,steering_speed,temperature_out,hour,month,light_sensor_rear,light_sensor_front,temperature_in,KBI_speed,soc,...,session,Label,time_second,distance_driven,weekday,car_id,ess_status,current_drive_mode,current_clima_mode,current_media_source
0,0.000000,0.000000,0.0,20,9,0.0,0.000000,0.0,0.00,0.00,...,7.0,NaN,0.0,0.0,2,1,0,car/driveMode/0.0,clima/AC/Auto,media/selectedSource/unavailable
1,0.000000,0.000000,0.0,20,9,0.0,0.000000,0.0,0.00,48.00,...,7.0,NaN,1.0,10845.0,2,1,0,car/driveMode/0.0,clima/AC/Auto,media/selectedSource/unavailable
2,0.000000,0.000000,25.0,20,9,0.0,0.000000,0.0,0.00,48.00,...,7.0,NaN,2.0,10845.0,2,1,0,car/driveMode/0.0,clima/AC/Auto,media/selectedSource/unavailable
3,0.000000,0.000000,25.0,20,9,0.0,0.000000,27.0,0.00,48.00,...,7.0,NaN,3.0,10845.0,2,1,0,car/driveMode/0.0,clima/AC/Auto,media/selectedSource/unavailable
4,0.000000,0.000000,25.0,20,9,0.0,0.000000,27.0,0.00,48.00,...,7.0,NaN,3.0,10845.0,2,1,0,car/driveMode/0.0,clima/AC/Auto,media/selectedSource/unavailable
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10124879,53.755324,0.000000,13.5,17,3,17600.0,6126.000000,28.5,92.48,22.25,...,5399.0,NaN,5360.0,146.0,4,2,0,car/driveMode/0.0,clima/AC/Auto,media/selectedSource/unavailable
10124880,50.000000,0.000000,13.5,17,3,17200.0,5920.781157,28.5,93.12,22.20,...,5399.0,NaN,5361.0,146.0,4,2,0,car/driveMode/0.0,clima/AC/Auto,media/selectedSource/unavailable
10124881,56.226313,15.000000,13.5,17,3,18000.0,5699.908827,28.5,93.44,22.20,...,5399.0,NaN,5362.0,146.0,4,2,0,car/driveMode/0.0,clima/AC/Auto,media/selectedSource/unavailable
10124882,60.000000,0.000000,13.5,17,3,17600.0,5437.218229,28.5,93.76,22.20,...,5399.0,NaN,5363.0,146.0,4,2,0,car/driveMode/0.0,clima/AC/Auto,media/selectedSource/unavailable


In [34]:
test_session = [387]
navi_status = []

# Iterate over unique sessions
for session in tqdm(data_media['session'].unique()):
    # Subset dataframe for the current session45
    df = data_media[data_media['session'] == session].copy().reset_index()
    
    # Initialize 'nav_guidance_status' column
    df['nav_guidance_status'] = 'navi_inactive'

    # Define labels related to navigation
    navi_labels = {'navi/Start/Address', 'navi/Start/Favorite'}
    
    # Check if navigation-related labels exist in the session
    if 'navi' in df['Label'].str.split('/').str[0].values:
        # Iterate through rows of the session dataframe
        for i, row in df.iterrows():
            label = row['Label']
            if label in navi_labels:
                # Update 'nav_guidance_status' to indicate active navigation
                df.loc[i:, 'nav_guidance_status'] = 'navi_active'
                break
    navi_status.append(df)

# Concatenate dataframes for all sessions
data_navi = pd.concat(navi_status, axis=0)

100%|██████████| 1704/1704 [01:14<00:00, 22.88it/s]


In [35]:
data_media.dropna()[data_media.dropna()['Label'].str.contains('ParkAssistant')].dropna()

,avg_irradiation,steering_speed,temperature_out,hour,month,light_sensor_rear,light_sensor_front,temperature_in,KBI_speed,soc,...,session,Label,time_second,distance_driven,weekday,car_id,ess_status,current_drive_mode,current_clima_mode,current_media_source
3744985,5.707657,0.0,9.5,10,12,3600.000000,1110.000000,25.5,0.00000,91.35,...,1659.0,car/Start/ParkAssistant,9674.0,28353.0,3,6,1,car/driveMode/0.0,clima/AC/Auto,media/selectedSource/unavailable
8594242,63.394196,0.0,23.5,14,6,19836.577804,5138.451333,25.0,25.84526,81.50,...,4589.0,car/Start/ParkAssistant,1265.0,28210.0,3,5,0,car/driveMode/0.0,clima/AC/Auto,media/selectedSource/unavailable


In [36]:
test_session = [387]
park_assistants_status = []

# Iterate over unique sessions
for session in tqdm(data_navi['session'].unique()):
    # Subset dataframe for the current session
    df = data_navi[data_navi['session'] == session].copy().reset_index()
    
    # Initialize 'nav_guidance_status' column
    df['proximity_to_parking_spot'] = 'no_parking_spot_closeby'

    # Define labels related to navigation
    park_assistant = {'car/Start/ParkAssistant'}

    # df.dropna()[df.dropna()['Label'].str.contains('ParkAssistant')].dropna()
    # Check if navigation-related labels exist in the session
    if df['Label'].str.contains('ParkAssistant').any():
        print(session, 'i am in')
        # Iterate through rows of the session dataframe
        for i, row in df.iterrows():
            label = row['Label']
            if label in park_assistant:
                # Update 'nav_guidance_status' to indicate active navigation
                df.loc[i-window:i+window, 'proximity_to_parking_spot'] = 'parking_spot_closeby'
                break
    park_assistants_status.append(df)

# Concatenate dataframes for all sessions
data_parkassistant = pd.concat(park_assistants_status, axis=0)

 36%|███▌      | 611/1704 [00:07<00:15, 72.78it/s]

1659.0 i am in


 84%|████████▍ | 1438/1704 [00:19<00:03, 83.37it/s]

4589.0 i am in


100%|██████████| 1704/1704 [00:22<00:00, 74.88it/s]


In [37]:
data_parkassistant[data_parkassistant['session']==1659.0]['proximity_to_parking_spot'].unique()

array(['no_parking_spot_closeby', 'parking_spot_closeby'], dtype=object)

In [38]:
data_media.dropna()[data_media.dropna()['Label'].str.contains('phone')].dropna()

,avg_irradiation,steering_speed,temperature_out,hour,month,light_sensor_rear,light_sensor_front,temperature_in,KBI_speed,soc,...,session,Label,time_second,distance_driven,weekday,car_id,ess_status,current_drive_mode,current_clima_mode,current_media_source
34273,20.00000,0.000000,13.0,11,9,12000.000000,3576.000000,24.5,11.691761,70.10,...,46.0,phone/goTo/Favorite,1871.0,11524.0,4,1,0,car/driveMode/0.0,clima/AC/Auto,media/selectedSource/unavailable
44552,0.00000,53.388109,8.5,15,12,2400.000000,738.000000,24.5,15.040000,37.45,...,150.0,phone/Start/CarPlay,1835.0,13795.0,1,1,0,car/driveMode/0.0,clima/AC/Auto,media/selectedSource/unavailable
144772,0.00000,203.264859,13.5,16,12,2800.000000,975.726155,20.5,6.400000,77.20,...,203.0,phone/goTo/Favorite,325.0,14713.0,3,1,0,car/driveMode/0.0,clima/AC/Auto,media/selectedSource/unavailable
224365,16.10772,0.000000,9.0,17,4,4000.000000,6126.000000,22.0,0.000000,25.05,...,385.0,phone/Start/AndroidAuto,88.0,22242.0,1,1,0,car/driveMode/0.0,clima/AC/Auto,media/selectedSource/unavailable
420006,40.00000,0.000000,12.0,12,4,16400.000000,4549.259395,17.0,0.000000,75.20,...,465.0,phone/goTo/Favorite,175.0,23564.0,2,1,0,car/driveMode/0.0,clima/AC/Auto,media/selectedSource/unavailable
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10058296,0.00000,0.000000,19.0,20,3,0.000000,0.000000,24.5,9.600000,97.90,...,5334.0,phone/goTo/Favorite,195.0,21588.0,0,2,0,car/driveMode/0.0,clima/AC/Auto,media/selectedSource/unavailable
10064080,0.00000,0.000000,9.5,19,3,400.000000,420.000000,16.5,122.880000,79.05,...,5339.0,phone/goTo/Favorite,553.0,21666.0,1,2,0,car/driveMode/0.0,clima/AC/Auto,media/selectedSource/unavailable
10066875,20.00000,0.000000,0.5,8,3,1200.000000,3173.297789,14.5,74.240000,66.45,...,5341.0,phone/goTo/Favorite,796.0,21715.0,2,2,0,car/driveMode/0.0,clima/AC/Auto,media/selectedSource/unavailable
10117200,270.00000,0.000000,8.5,12,3,30834.067022,6126.000000,26.5,140.160000,56.30,...,5387.0,phone/goTo/Favorite,891.0,24.0,1,2,0,car/driveMode/0.0,clima/AC/Auto,media/selectedSource/unavailable


In [39]:
data_media.dropna()[data_media.dropna()['Label'].str.contains('phone')].dropna()['Label'].unique()

array(['phone/goTo/Favorite', 'phone/Start/CarPlay',
       'phone/Start/AndroidAuto', 'phone/Connect/NewDevice',
       'phone/Call/Favorite', 'phone/Call/PersonX'], dtype=object)

In [40]:
data_parkassistant

,level_0,index,avg_irradiation,steering_speed,temperature_out,hour,month,light_sensor_rear,light_sensor_front,temperature_in,...,time_second,distance_driven,weekday,car_id,ess_status,current_drive_mode,current_clima_mode,current_media_source,nav_guidance_status,proximity_to_parking_spot
0,0,0,0.000000,0.000000,0.0,20,9,0.0,0.000000,0.0,...,0.0,0.0,2,1,0,car/driveMode/0.0,clima/AC/Auto,media/selectedSource/unavailable,navi_inactive,no_parking_spot_closeby
1,1,1,0.000000,0.000000,0.0,20,9,0.0,0.000000,0.0,...,1.0,10845.0,2,1,0,car/driveMode/0.0,clima/AC/Auto,media/selectedSource/unavailable,navi_inactive,no_parking_spot_closeby
2,2,2,0.000000,0.000000,25.0,20,9,0.0,0.000000,0.0,...,2.0,10845.0,2,1,0,car/driveMode/0.0,clima/AC/Auto,media/selectedSource/unavailable,navi_inactive,no_parking_spot_closeby
3,3,3,0.000000,0.000000,25.0,20,9,0.0,0.000000,27.0,...,3.0,10845.0,2,1,0,car/driveMode/0.0,clima/AC/Auto,media/selectedSource/unavailable,navi_inactive,no_parking_spot_closeby
4,4,4,0.000000,0.000000,25.0,20,9,0.0,0.000000,27.0,...,3.0,10845.0,2,1,0,car/driveMode/0.0,clima/AC/Auto,media/selectedSource/unavailable,navi_inactive,no_parking_spot_closeby
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2066,2066,10124879,53.755324,0.000000,13.5,17,3,17600.0,6126.000000,28.5,...,5360.0,146.0,4,2,0,car/driveMode/0.0,clima/AC/Auto,media/selectedSource/unavailable,navi_inactive,no_parking_spot_closeby
2067,2067,10124880,50.000000,0.000000,13.5,17,3,17200.0,5920.781157,28.5,...,5361.0,146.0,4,2,0,car/driveMode/0.0,clima/AC/Auto,media/selectedSource/unavailable,navi_inactive,no_parking_spot_closeby
2068,2068,10124881,56.226313,15.000000,13.5,17,3,18000.0,5699.908827,28.5,...,5362.0,146.0,4,2,0,car/driveMode/0.0,clima/AC/Auto,media/selectedSource/unavailable,navi_inactive,no_parking_spot_closeby
2069,2069,10124882,60.000000,0.000000,13.5,17,3,17600.0,5437.218229,28.5,...,5363.0,146.0,4,2,0,car/driveMode/0.0,clima/AC/Auto,media/selectedSource/unavailable,navi_inactive,no_parking_spot_closeby


In [41]:
test_session = [387]
phone_status = []

data_parkassistant = data_parkassistant.sort_values(by=['session', 'datetime'])
data_parkassistant = data_parkassistant.reset_index(drop=True)
data_parkassistant = data_parkassistant.drop(columns=['level_0', 'index'])

for session in tqdm(data_parkassistant['session'].unique()):

    df = data_parkassistant[data_parkassistant['session'] == session].copy().reset_index()
    
    df['phone_status'] = 'unconnected'

    phone_labels = {'phone/goTo/Favorite', 'phone/Start/CarPlay',
       'phone/Start/AndroidAuto', 'phone/Connect/NewDevice',
       'phone/Call/Favorite', 'phone/Call/PersonX'}
    
    if df['Label'].str.contains('phone').any():

        for i, row in df.iterrows():
            label = row['Label']
            if label in phone_labels:
                df.loc[i-window:i, 'phone_status'] = 'phone_connected'
    phone_status.append(df)

data_phone_status = pd.concat(phone_status, axis=0)

100%|██████████| 1704/1704 [02:42<00:00, 10.50it/s]


In [42]:
data_phone_status[data_phone_status['session']==46]['phone_status'].unique()

array(['unconnected', 'phone_connected'], dtype=object)

In [43]:
test_session = [387]
bluetooth_device_status = []

data_phone_status = data_phone_status.sort_values(by=['session', 'datetime'])
data_phone_status = data_phone_status.reset_index(drop=True)
data_phone_status = data_phone_status.drop(columns=['index'])

for session in tqdm(data_phone_status['session'].unique()):
    df = data_phone_status[data_phone_status['session'] == session].copy().reset_index()
    df['bluetooth_connected'] = 'bluetooth_unconnected'
    phone_labels = {'media/selectedSource/Bluetooth'}
    if df['Label'].str.contains('Bluetooth|phone').any():
        for i, row in df.iterrows():
            label = row['Label']
            if label in phone_labels:
                df.loc[i-window:, 'bluetooth_connected'] = 'bluetooth_connected'
    bluetooth_device_status.append(df)

data_bluetooth_status = pd.concat(bluetooth_device_status, axis=0)

100%|██████████| 1704/1704 [03:55<00:00,  7.24it/s]


In [44]:
south_germany_season_mapping = {
    1: 'Winter',
    2: 'Winter',
    3: 'Spring',
    4: 'Spring',
    5: 'Spring',
    6: 'Summer',
    7: 'Summer',
    8: 'Summer',
    9: 'Autumn',
    10: 'Autumn',
    11: 'Autumn',
    12: 'Winter'
}

data_bluetooth_status['season'] = data_bluetooth_status['month'].map(south_germany_season_mapping)

In [45]:
data_bluetooth_status.dropna()[data_bluetooth_status.dropna()['Label'].str.contains('CarPlay')].dropna()

,index,avg_irradiation,steering_speed,temperature_out,hour,month,light_sensor_rear,light_sensor_front,temperature_in,KBI_speed,...,car_id,ess_status,current_drive_mode,current_clima_mode,current_media_source,nav_guidance_status,proximity_to_parking_spot,phone_status,bluetooth_connected,season
1835,44552,0.0,53.388109,8.5,15,12,2400.000000,738.000000,24.5,15.04,...,1,0,car/driveMode/0.0,clima/AC/Auto,media/selectedSource/unavailable,navi_inactive,no_parking_spot_closeby,phone_connected,bluetooth_unconnected,Winter
4158,4424458,0.0,0.000000,10.0,14,2,2400.000000,4656.000000,25.0,0.00,...,6,0,car/driveMode/3.0,clima/AC/Auto,media/selectedSource/unavailable,navi_inactive,no_parking_spot_closeby,unconnected,bluetooth_unconnected,Winter
4159,4424459,0.0,0.000000,10.0,14,2,2400.000000,4656.000000,25.0,0.00,...,6,0,car/driveMode/3.0,clima/AC/Auto,media/selectedSource/unavailable,navi_inactive,no_parking_spot_closeby,unconnected,bluetooth_unconnected,Winter
240,5206758,0.0,3.129767,15.0,21,9,0.000000,0.000000,19.5,79.36,...,4,0,car/driveMode/0.0,clima/AC/Auto,media/selectedSource/unavailable,navi_inactive,no_parking_spot_closeby,phone_connected,bluetooth_unconnected,Autumn
2033,5823679,10.0,0.000000,8.5,15,11,5200.000000,721.081688,15.0,0.00,...,4,0,car/driveMode/0.0,clima/AC/Auto,media/selectedSource/unavailable,navi_inactive,no_parking_spot_closeby,unconnected,bluetooth_unconnected,Autumn
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2106,10041462,50.0,0.000000,13.0,10,3,17200.000000,6126.000000,25.0,40.64,...,2,0,car/driveMode/0.0,clima/AC/Auto,media/selectedSource/unavailable,navi_active,no_parking_spot_closeby,unconnected,bluetooth_unconnected,Spring
3951,10043307,40.0,0.000000,11.5,10,3,15767.936863,4683.847576,27.0,94.72,...,2,0,car/driveMode/0.0,clima/AC/Auto,media/selectedSource/unavailable,navi_active,no_parking_spot_closeby,unconnected,bluetooth_unconnected,Spring
394,10048979,110.0,5.000000,17.0,13,3,37386.293552,6126.000000,27.0,56.96,...,2,0,car/driveMode/0.0,clima/AC/Auto,media/selectedSource/unavailable,navi_inactive,no_parking_spot_closeby,unconnected,bluetooth_unconnected,Spring
116,10055037,0.0,5.000000,19.5,18,3,2000.000000,2625.849453,26.5,38.72,...,2,0,car/driveMode/0.0,clima/AC/Auto,media/selectedSource/unavailable,navi_inactive,no_parking_spot_closeby,unconnected,bluetooth_unconnected,Spring


In [46]:
test_session = [387]
phone_os = []

data_bluetooth_status = data_bluetooth_status.sort_values(by=['session', 'datetime'])
data_bluetooth_status = data_bluetooth_status.reset_index(drop=True)
data_bluetooth_status = data_bluetooth_status.drop(columns=['index'])


phone_labels = {'phone/Start/CarPlay', 'media/selectedSource/CarPlay',  'phone/Start/AndroidAuto'}
Android = {'phone/Start/AndroidAuto'}
carplay = {'phone/Start/CarPlay', 'media/selectedSource/CarPlay'}
others = {'phone/goTo/Favorite', 'phone/Connect/NewDevice',
       'phone/Call/Favorite', 'phone/Call/PersonX', 'media/selectedSource/Bluetooth', }

phone_os = []

for session in tqdm(data_bluetooth_status['session'].unique()):
    df = data_bluetooth_status[data_bluetooth_status['session'] == session].copy()
    df['phone_os'] = 'unconnected'
    if df['Label'].str.contains('Bluetooth|phone|AndroidAuto|CarPlay').any():
        if df['Label'].str.contains('AndroidAuto').any():
            df['phone_os'] = 'Android'
        elif df['Label'].str.contains('CarPlay').any():
            df['phone_os'] = 'ios'
        else:
            df['phone_os'] = 'unknown'
    phone_os.append(df)

data_phone_os = pd.concat(phone_os, axis=0)

100%|██████████| 1704/1704 [00:21<00:00, 77.73it/s]


In [47]:
data_phone_os[data_phone_os['session']==16]

,avg_irradiation,steering_speed,temperature_out,hour,month,light_sensor_rear,light_sensor_front,temperature_in,KBI_speed,soc,...,ess_status,current_drive_mode,current_clima_mode,current_media_source,nav_guidance_status,proximity_to_parking_spot,phone_status,bluetooth_connected,season,phone_os
4187,0.0,0.0,0.0,17,9,0.0,0.0,0.0,0.0,0.00,...,0,car/driveMode/0.0,clima/AC/Auto,media/selectedSource/unavailable,navi_inactive,no_parking_spot_closeby,unconnected,bluetooth_unconnected,Autumn,unknown
4188,0.0,0.0,0.0,17,9,0.0,24.0,0.0,0.0,0.00,...,0,car/driveMode/0.0,clima/AC/Auto,media/selectedSource/unavailable,navi_inactive,no_parking_spot_closeby,unconnected,bluetooth_unconnected,Autumn,unknown
4189,0.0,0.0,22.0,17,9,0.0,24.0,0.0,0.0,69.30,...,0,car/driveMode/0.0,clima/AC/Auto,media/selectedSource/unavailable,navi_inactive,no_parking_spot_closeby,unconnected,bluetooth_unconnected,Autumn,unknown
4190,0.0,0.0,22.0,17,9,0.0,24.0,26.0,0.0,69.30,...,0,car/driveMode/0.0,clima/AC/Auto,media/selectedSource/unavailable,navi_inactive,no_parking_spot_closeby,unconnected,bluetooth_unconnected,Autumn,unknown
4191,0.0,0.0,22.0,17,9,0.0,24.0,26.0,0.0,69.30,...,0,car/driveMode/0.0,clima/AC/Auto,media/selectedSource/unavailable,navi_inactive,no_parking_spot_closeby,unconnected,bluetooth_unconnected,Autumn,unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7623,0.0,0.0,20.5,18,9,3200.0,852.0,26.5,0.0,56.65,...,0,car/driveMode/0.0,clima/AC/Auto,media/selectedSource/unavailable,navi_inactive,no_parking_spot_closeby,unconnected,bluetooth_connected,Autumn,unknown
7624,0.0,0.0,20.5,18,9,3200.0,852.0,26.5,0.0,56.65,...,0,car/driveMode/0.0,clima/AC/Auto,media/selectedSource/unavailable,navi_inactive,no_parking_spot_closeby,unconnected,bluetooth_connected,Autumn,unknown
7625,0.0,0.0,20.5,18,9,3200.0,852.0,26.5,0.0,56.65,...,0,car/driveMode/0.0,clima/AC/Auto,media/selectedSource/unavailable,navi_inactive,no_parking_spot_closeby,unconnected,bluetooth_connected,Autumn,unknown
7626,0.0,0.0,20.5,18,9,3200.0,852.0,26.5,0.0,56.65,...,0,car/driveMode/0.0,clima/AC/Auto,media/selectedSource/unavailable,navi_inactive,no_parking_spot_closeby,unconnected,bluetooth_connected,Autumn,unknown


In [48]:
import torch
torch.tensor([[1,2]])

tensor([[1, 2]])

In [49]:
data_phone_os.dropna()[data_phone_os.dropna()['Label'].str.contains('NewDevice')].dropna()

,avg_irradiation,steering_speed,temperature_out,hour,month,light_sensor_rear,light_sensor_front,temperature_in,KBI_speed,soc,...,ess_status,current_drive_mode,current_clima_mode,current_media_source,nav_guidance_status,proximity_to_parking_spot,phone_status,bluetooth_connected,season,phone_os
2161654,0.000000,0.000000,21.0,20,9,0.0,0.0,23.5,0.0,61.00,...,0,car/driveMode/0.0,clima/AC/Auto,media/selectedSource/unavailable,navi_inactive,no_parking_spot_closeby,phone_connected,bluetooth_connected,Autumn,unknown
3744515,38.484954,0.000000,9.5,10,12,3600.0,1314.0,26.5,0.0,91.50,...,0,car/driveMode/0.0,clima/AC/Auto,media/selectedSource/unavailable,navi_active,no_parking_spot_closeby,phone_connected,bluetooth_connected,Winter,unknown
9748092,0.000000,3.523319,3.0,19,1,0.0,0.0,21.5,36.8,53.55,...,0,car/driveMode/0.0,clima/AC/Auto,media/selectedSource/unavailable,navi_inactive,no_parking_spot_closeby,phone_connected,bluetooth_unconnected,Winter,unknown


In [50]:
new_bluetooth_device = []

for session in tqdm(data_phone_os['session'].unique()):
    df = data_phone_os[data_phone_os['session'] == session].reset_index()
    df['new_bluetooth_device_to_pair'] = 'no bluetooth device to pair'
    
    new_device_indices = df[df['Label'].str.contains('NewDevice', na=False)].index
    if not new_device_indices.empty:
        start_index = max(0, new_device_indices[0] - window)  # Calculate start index
        end_index = new_device_indices[-1]  # Calculate end index
        print(start_index, end_index)
        # Update the new_bluetooth_device_to_pair column for the specified range
        df.loc[start_index:end_index, 'new_bluetooth_device_to_pair'] = 'new bluetooth device available'
    
    new_bluetooth_device.append(df)

# Concatenate all dataframes in new_bluetooth_device list
data_new_bluetooth_device = pd.concat(new_bluetooth_device, axis=0)

 23%|██▎       | 397/1704 [00:05<00:16, 81.03it/s]

0 38


 37%|███▋      | 627/1704 [00:08<00:15, 71.33it/s]

9102 9202


 96%|█████████▌| 1640/1704 [00:22<00:00, 66.21it/s]

268 368


100%|██████████| 1704/1704 [00:23<00:00, 73.99it/s]


In [51]:
data_new_bluetooth_device

,index,avg_irradiation,steering_speed,temperature_out,hour,month,light_sensor_rear,light_sensor_front,temperature_in,KBI_speed,...,current_drive_mode,current_clima_mode,current_media_source,nav_guidance_status,proximity_to_parking_spot,phone_status,bluetooth_connected,season,phone_os,new_bluetooth_device_to_pair
0,0,0.000000,0.000000,0.0,20,9,0.0,0.000000,0.0,0.00,...,car/driveMode/0.0,clima/AC/Auto,media/selectedSource/unavailable,navi_inactive,no_parking_spot_closeby,unconnected,bluetooth_unconnected,Autumn,unknown,no bluetooth device to pair
1,1,0.000000,0.000000,0.0,20,9,0.0,0.000000,0.0,0.00,...,car/driveMode/0.0,clima/AC/Auto,media/selectedSource/unavailable,navi_inactive,no_parking_spot_closeby,unconnected,bluetooth_unconnected,Autumn,unknown,no bluetooth device to pair
2,2,0.000000,0.000000,25.0,20,9,0.0,0.000000,0.0,0.00,...,car/driveMode/0.0,clima/AC/Auto,media/selectedSource/unavailable,navi_inactive,no_parking_spot_closeby,unconnected,bluetooth_unconnected,Autumn,unknown,no bluetooth device to pair
3,3,0.000000,0.000000,25.0,20,9,0.0,0.000000,27.0,0.00,...,car/driveMode/0.0,clima/AC/Auto,media/selectedSource/unavailable,navi_inactive,no_parking_spot_closeby,unconnected,bluetooth_unconnected,Autumn,unknown,no bluetooth device to pair
4,4,0.000000,0.000000,25.0,20,9,0.0,0.000000,27.0,0.00,...,car/driveMode/0.0,clima/AC/Auto,media/selectedSource/unavailable,navi_inactive,no_parking_spot_closeby,unconnected,bluetooth_unconnected,Autumn,unknown,no bluetooth device to pair
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2066,10124879,53.755324,0.000000,13.5,17,3,17600.0,6126.000000,28.5,92.48,...,car/driveMode/0.0,clima/AC/Auto,media/selectedSource/unavailable,navi_inactive,no_parking_spot_closeby,unconnected,bluetooth_unconnected,Spring,unconnected,no bluetooth device to pair
2067,10124880,50.000000,0.000000,13.5,17,3,17200.0,5920.781157,28.5,93.12,...,car/driveMode/0.0,clima/AC/Auto,media/selectedSource/unavailable,navi_inactive,no_parking_spot_closeby,unconnected,bluetooth_unconnected,Spring,unconnected,no bluetooth device to pair
2068,10124881,56.226313,15.000000,13.5,17,3,18000.0,5699.908827,28.5,93.44,...,car/driveMode/0.0,clima/AC/Auto,media/selectedSource/unavailable,navi_inactive,no_parking_spot_closeby,unconnected,bluetooth_unconnected,Spring,unconnected,no bluetooth device to pair
2069,10124882,60.000000,0.000000,13.5,17,3,17600.0,5437.218229,28.5,93.76,...,car/driveMode/0.0,clima/AC/Auto,media/selectedSource/unavailable,navi_inactive,no_parking_spot_closeby,unconnected,bluetooth_unconnected,Spring,unconnected,no bluetooth device to pair


In [52]:
#find time delta between interactions
interaction_time_delta = []
data_new_bluetooth_device['datetime'] = pd.to_datetime(data_new_bluetooth_device['datetime'])
for session in tqdm(data_new_bluetooth_device['session'].unique()):
    df = data_new_bluetooth_device[data_new_bluetooth_device['session'] == session].reset_index()
    df['timedelta'] = 0
    indices = df.index[df['Label'].notna()]
    # timedeltas = df.loc[indices, 'datetime'].diff()
    # print(timedeltas.values)
    # df['timedelta'] = pd.NaT
    # df.loc[indices[0:], 'timedelta'] = timedeltas.values
    df.loc[indices, 'timedelta'] = df.loc[indices, 'datetime'].diff().dt.total_seconds() / 60
    df['timedelta'] = df['timedelta'].fillna(0)
    interaction_time_delta.append(df)
data_interaction_time_delta = pd.concat(interaction_time_delta, axis=0)

100%|██████████| 1704/1704 [00:24<00:00, 70.24it/s]


In [53]:
data_interaction_time_delta[data_interaction_time_delta['session']==16][['session', 'timedelta','Label']].dropna()

,session,timedelta,Label
233,16.0,0.000000,media/selectedSource/Radio
234,16.0,0.000000,media/selectedSource/Radio
848,16.0,10.216667,media/selectedSource/Bluetooth
2151,16.0,21.733333,media/selectedSource/Radio
3297,16.0,19.116667,media/selectedSource/Radio


In [54]:
data_interaction_time_delta.to_csv('../data/05_Interaction_Sequences/context_newfeat.csv', index=False)